# 拍點音符紀錄器 for 和弦判斷
- 附點< dots >, 多連音< Tuplet >, 各種拍子、變拍< TimeSig >已處理


In [19]:
import numpy as np
from decimal import *

def convert_chord(tu):
    origin = [0,0,0,0,0,0,0,0,0,0,0,0]
    for i in tu:
        origin[i%12]=1
    return tuple(origin)

# chord = {
#     convert((0,4,7)):'Major',convert((0,3,7)):'minor',
#     convert((0,4,8)):'aug',convert((0,3,6)):'dim',
#     convert((0,4,7,11)):'Major7',convert((0,4,7,10)):'dom7',
#     convert((0,3,7,10)):'minor7',convert((0,3,6,10)):'m7flat5',
#     convert((0,2,7)):'2',convert((0,5,7)):'4',
#     convert((0,2,4,7,10)):'9'
# }
chord = {
    convert((0,4,7)):'C',convert((2,5,9)):'Dm',convert((4,7,11)):'Em',
    convert((0,5,9)):'F',convert((2,7,11)):'G',convert((0,4,9)):'Am',
    convert((2,5,10)):'Bb'
}

chord_np = np.array(chord.keys())

key = {
    0:'C',1:'C-sharp',2:'D',3:'D-sharp',4:'E',5:'F',6:'F-sharp',7:'G',8:'G-sharp',9:'A',10:'A-sharp',11:'B'
} # pitch%12

time = {
    'measure':None,
    'whole':Decimal(1),
    'half':Decimal(1)/2,
    'quarter':Decimal(1)/4,
    'eighth':Decimal(1)/8,
    '16th':Decimal(1)/16,
    '32nd':Decimal(1)/32,
    '64th':Decimal(1)/64,
    '128th':Decimal(1)/128
}

def ifDots(tag):
    if not tag.find('dots'):
        multi = 1
    else:
        dots = tag.find('dots').text
        if dots == '1':
            multi = 1.5
        elif dots == '2':
            multi = 1.75
        elif dots == '3':
            multi = 1.875
    return Decimal(multi)

In [14]:
from bs4 import BeautifulSoup as bs
import sys

#Basket Case是C小調(聽起來像Eb大調)
f = open('C:/Users/BigData/Desktop/Love Story.mscx','r')
music = bs(f.read(),'xml')
f.close()
staff1TimeSig = music.select_one('Score > Staff:nth-of-type(1) TimeSig')
sigN = Decimal(staff1TimeSig.find('sigN').text)
sigD = Decimal(staff1TimeSig.find('sigD').text)
time['measure'] = sigN/sigD
l = []
dic = {}
for staff in music.select('Score > Staff'):
    time_acc = 0
    tupletID = ''
    tupletRatio = 1
    for tag in staff.find_all(['Rest','Chord','Tuplet','TimeSig']):

        #Additional <track> in every measure is ignored
        if (tag.name == 'Chord' or tag.name == 'Rest') and not tag.find('track'):
            multi = ifDots(tag)
            if tag.find('Tuplet') and tag.find('Tuplet').text == tupletID:
                multi *= tupletRatio
            duTime = Decimal(time[tag.select('durationType')[0].text])
            time_acc += duTime * multi
            if tag.name == 'Rest':
                l.append((time_acc, 0))
            else:
                pitches = tag.select('pitch')
                
                #目前音高為手動修改
                addlist = [time_acc] + [int(a.text)-3 for a in pitches]
                l.append(tuple(addlist))
        
        #Only take <Tuplet> with id to avoid children
        #<Tuplet> inside <Chord> or <Rest>
        elif tag.name == 'Tuplet' and tag.get('id'):
            tupletID = tag['id']
            normalNotes = Decimal(tag.find('normalNotes').text)
            actualNotes = Decimal(tag.find('actualNotes').text)
            tupletRatio = normalNotes / actualNotes
        
        elif tag.name == 'TimeSig':
            sigN = Decimal(tag.find('sigN').text)
            sigD = Decimal(tag.find('sigD').text)
            if sigN/sigD != time['measure']:
                time['measure'] = sigN/sigD
    dic.update({'staff_'+staff.get('id') : l})
    l = []

In [15]:
for i, staff in enumerate(dic, 1):
    print '[Staff #' + str(i) + ']'
    print '-------------------------'
    for e in dic[staff]:
        print e
    print '-------------------------'

[Staff #1]
-------------------------
(Decimal('0.125'), 0)
(Decimal('0.250'), 59)
(Decimal('0.500'), 59, 75)
(Decimal('0.625'), 59, 75)
(Decimal('0.875'), 71)
(Decimal('1.000'), 71)
(Decimal('2.000'), 0)
(Decimal('2.125'), 0)
(Decimal('2.250'), 59)
(Decimal('2.500'), 59, 75)
(Decimal('2.625'), 59, 75)
(Decimal('2.875'), 71)
(Decimal('3.000'), 71)
(Decimal('4.000'), 0)
(Decimal('4.125'), 0)
(Decimal('4.250'), 59)
(Decimal('4.500'), 59, 75)
(Decimal('4.625'), 59, 75)
(Decimal('4.875'), 71)
(Decimal('5.000'), 71)
(Decimal('6.000'), 0)
(Decimal('6.125'), 0)
(Decimal('6.250'), 59)
(Decimal('6.500'), 59, 75)
(Decimal('6.625'), 59, 75)
(Decimal('6.875'), 71)
(Decimal('7.000'), 71)
(Decimal('8.000'), 0)
(Decimal('9.000'), 0)
(Decimal('10.000'), 0)
(Decimal('11.000'), 0)
(Decimal('12.000'), 0)
(Decimal('13.000'), 0)
(Decimal('14.000'), 0)
(Decimal('15.000'), 0)
(Decimal('15.125'), 0)
(Decimal('15.250'), 59)
(Decimal('15.500'), 59, 75)
(Decimal('15.625'), 59, 75)
(Decimal('15.875'), 71)
(Decimal

# 以下未修改

In [16]:
# 第一步，判斷每個staff，有幾成的比例，使用和弦   by Henry
# 1-1. 目前預設，只要大於70%，就紀錄該staff
# 1-2. pitch只要同時有2音以上，就當作和弦

pitch_count = 0
percent = 0.3
dominant_percent_staff_list = []

for staff in dic:
    pitch_count = 0
    for tempo_pitch in dic[staff]:
        if len(tempo_pitch) > 3 :
            pitch_count += 1
    if float(pitch_count) / float(len(dic[staff])) >= percent:
        dominant_percent_staff_list.append(staff)
print dominant_percent_staff_list

#測試區

# for staff in dominant_percent_staff_list:
#     print staff
# print dic['staff_4']
# print type(dic)
# for staff in dic:
#     print staff
# for tempo_pitch in dic[staff]:
#     print tempo_pitch
#     print len(tempo_pitch)
# print pitch_count
# print type(pitch_count)
# print len(dic['staff_2'])
# print type(len(dic['staff_2']))
# if float(pitch_count) / float(len(dic['staff_2'])) >= 0.3:
#     dominant_percent_staff_list.append('staff_2')
#     print dominant_percent_staff_list

[u'staff_4', u'staff_2', u'staff_11']
[u'staff_4', u'staff_2', u'staff_11']


In [17]:
import numpy as np
import math
def nearest(nparray, pt):
    n = [math.sqrt(sum((i-pt)**2)) for i in nparray]
    mindis = min(n)
    idx = n.index(mindis)
    return nparray[idx]


In [54]:
chord_dic = {}
for staff in dominant_percent_staff_list:
    chord_gogo = []
    count = 0
    begin = 0
        
    while(count < dic[staff][-1][0]):
        while(count < dic[staff][begin][0]):
            count += 1.0/2
            if dic[staff][begin][1] == 0:
                chord_gogo.append(set([0]))
            else:
                now = [pit%12 for pit in dic[staff][begin][1:]]
#                 closest = nearest(chord_np,convert_chord(tuple(now)))  
#                 chord_gogo.append(chord[tuple(closest)])
                chord_gogo.append(set(now))

        begin += 1
    chord_dic.update({staff : chord_gogo})

In [58]:
# for staff in chord_dic:
#     print staff, chord_dic[staff]
all_chord =  tuple(chord_dic.values())
combined = map(lambda *args: reduce(lambda x,y: x|y,args),*all_chord)

In [61]:
for a in combined:
    closest = nearest(chord_np,convert_chord(tuple(a)))
    print chord[tuple(closest)]

G
G
G
G
A
G
A
A
G
G
G
G
A
A
A
A
G
G
G
G
A
A
A
A
G
G
G
G
A
A
A
A
G
G
G
G
A
A
A
A
G
G
G
G
A
A
A
A
A
A
A
A
A
A
G
G
A
A
A
A
A
A
A
A
G
G
G
G
G
G
A
G
G
G
G
G
Em
Em
Em
A
G
G
G
G
G
G
G
G
A
A
A
A
G
G
G
G
A
A
A
A
A
A
A
A
A
A
G
G
A
A
A
A
A
A
A
A
G
G
G
G
G
G
A
G
G
G
G
G
Em
Em
Em
Em
G
G
G
G
G
G
A
G
G
G
G
G
Em
Em
Em
Em
G
G
G
G
A
A
A
A
A
A
A
A
A
A
G
A
G
G
G
G
G
G
A
A
G
Em
A
G
G
G
A
A
G
G
G
G
A
A
A
A
A
A
A
A
A
A
A
A
F
F
F
F
A
A
A
A
F
F
F
F
G
G
G
G
F
F
F
F
A
A
A
A
F
F
F
F
G
G
G
G
A
F


In [57]:
s1 = set([2,4,5])
s2 = set([1,2,3])
s3 = set([3,3,4])
a = (s1,s2,s3)
print reduce(lambda x,y: x|y,a)
# print map(lambda *args: sum(args),*a)

set([1, 2, 3, 4, 5])


In [147]:
c = []
for i in range(1,len(chord_dic['staff_1'])+1):
    if i%8 == 0:
        c.append(chord_dic['staff_1'][i-1])
        print c
        c = []
    else:
        c.append(chord_dic['staff_1'][i-1])


[0, 0, 0, 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 'F', 'F', 'C', 'C']
['G', 'G', 'G', 'G', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'C', 'C', 'C', 'F', 'F', 'G', 'G']
['C', 'F', 'A', 'A', 'F', 'F', 'G', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
[0, 0, 0, 'F', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['C', 'C', 'G', 'G', 'A', 'A', 'Em', 'Em']
['F', 0, 'G', 'C', 'A', 0, 'F', 0]
[0, 0, 'C', 'C', 'G', 'G', 'G', 'G']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'C', 'C', 'C']
['F', 'F', 'G', 'G', 'C', 'F', 'A', 'A']
['F', 'F', 'G', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'C', 'G', 'A', 'G']
['C', 'G', 'A', 'G', 'F', 'F', 'F', 'F']
['G', 'G', 'G', 0, 'C', 'C', 'G', 'G']
['A', 'A', 'Em', 'Em', 0, 0, 'F', 'C']
[0, 0, 0, 0, 0, 0, 'C', 'C']
['G', 'G', 'G', 'G', 'C', 'C', 'G', 'G']
['A', 'A', 'Em'

In [48]:
l1 = [2,4,5]
l2 = [1,2,3]
l3 = [3,3,4]
a = (l1,l2,l3)
print map(lambda *args: sum(args),*a)

[6, 9, 12]


In [70]:
chord_gogo = []
count = 0
begin = 0
while(count<l2[-1][0]):
    while(count < l2[begin][0]):
        count += 1.0/2
        if l2[begin][1] == 0:
            chord_gogo.append(0)
        else:
            chord_gogo.append(key[l2[begin][1]%12])
    begin += 1

[5, 1, 3]


In [ ]:
now = [pit%12 for pit in dic[staff][begin][1:]]
                for pitch in now:
                    poss_key = key[pitch]
                    det = [(n-pitch)%12 for n in now]
                    vec = convert_chord(tuple(det))
                    poss_chord = nearest(chord_np,vec)[0]